In [ ]:
from pathlib import Path

from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt

from topostats.io import LoadScans
from topostats.filters import Filters
from topostats.plotting import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()

from cropping import interactive_crop_files

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/tobi-dna-damage/")
assert base_dir.exists()
raw_data_dir = base_dir / "data-spm"
assert raw_data_dir.exists()

files = list(raw_data_dir.glob("*.spm"))
print(f"found {len(files)} files")

loadscans = LoadScans(img_paths=files, channel="Height")
loadscans.get_data()
img_dict = loadscans.img_dict
clear_output()
print(f"loaded {len(img_dict)} images")

In [ ]:
# flatten images
for raw_image_filename, raw_image_data in img_dict.items():
    raw_image = raw_image_data["image_original"]
    p2nm = raw_image_data["pixel_to_nm_scaling"]

    filters = Filters(
        image=raw_image,
        filename=raw_image_filename,
        pixel_to_nm_scaling=p2nm,
        row_alignment_quantile=0.5,
        threshold_method="absolute",
        otsu_threshold_multiplier=None,
        threshold_std_dev=None,
        threshold_absolute={
            "above": 1.0,
            "below": -1.0,
        },
        gaussian_size=1.0,
        gaussian_mode="nearest",
        remove_scars={
            "run": False,
        },
    )
    filters.filter_image()

    flattened_image = filters.images["gaussian_filtered"]
    img_dict[raw_image_filename]["image_flattened"] = flattened_image

In [ ]:
for image_filename, image_data in img_dict.items():
    print(image_filename)
    flattened_image = image_data["image_flattened"]
    p2nm = image_data["pixel_to_nm_scaling"]

    fig, ax = plt.subplots()
    zrange = [-2, 4]
    ax.imshow(flattened_image, cmap=cmap, clim=zrange)
    ax.set_title(image_filename)
    plt.show()
clear_output()

In [ ]:
crop_dir = base_dir / "cropped_images"
crop_dir.mkdir(exist_ok=True)

# create list of images
images = []
for image_filename, image_data in img_dict.items():
    flattened_image = image_data["image_flattened"]
    p2nm = image_data["pixel_to_nm_scaling"]
    images.append(flattened_image)

interactive_crop_files(images=images, crop_output_dir=crop_dir)